In [1]:
import os
import sys
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
import pickle
import subprocess
sys.path.insert(0, "C:/workspace/data-science//libs")
from simulation_core import Simulation
import big_numbers

pd.set_option('display.max_columns', None)

wbtc_usdc_sim = Simulation(experiment_id=1, x_name='WBTC', y_name='USDC', window_size=24,
                          base_dir=fr'{os.getcwd()}\data\real_transactions\experiment_')

86400 24 3600


In [2]:
# wbtc_usdc_sim.run_simulation(isBurnAvailable = True, isMintAvailable = True)    # passed, experiment 1

In [3]:
wbtc_usdc_swaps_mitigation_on_df, wbtc_usdc_mints_mitigation_on_df, wbtc_usdc_burns_mitigation_on_df = wbtc_usdc_sim.read_simulation(1)

In [4]:
def js_mitigator_check(record):   
    result = subprocess.run([
        "node", "main.js",
        "--balance_0", str(big_numbers.expand_to_18_decimals(record["reserve_X_before"])), 
        "--balance_1", str(big_numbers.expand_to_18_decimals(record["reserve_Y_before"])), 
        "--tokenIn", record["token_in"], 
        "--tokenOut", record["token_out"], 
        "--amountIn", str(big_numbers.expand_to_18_decimals(record["token_in_amount"])), 
        "--amountOut", str(big_numbers.expand_to_18_decimals(record["token_out_amount"])),
        "--oracleOut", str(big_numbers.expand_to_18_decimals(record["oracle_amount_out"])
                            ) if record["oracle_amount_out"] != np.nan else "undefined"
    ], stdout=subprocess.PIPE)
    record['js_mitigation_status'] = result.stdout.decode('ascii').split("\n")[0]
        
    return record

In [5]:
len(wbtc_usdc_swaps_mitigation_on_df)

71648

In [6]:
# smaller_part = wbtc_usdc_swaps_mitigation_on_df.head(10000)
# smaller_part["js_mitigation_status"] = np.zeros(len(smaller_part))
# smaller_part = smaller_part.apply(js_mitigator_check, axis=1)
# smaller_part

In [7]:
# smaller_part[smaller_part['js_mitigation_status'] == "Error: Out value too far from Oracle"]

In [8]:
# smaller_part[smaller_part['status'] == "BLOCKED_BY_VOLATILITY_MITIGATION"]

In [9]:
wbtc_usdc_swaps_mitigation_on_df["js_mitigation_status"] = np.zeros(len(wbtc_usdc_swaps_mitigation_on_df))
wbtc_usdc_swaps_mitigation_on_df = wbtc_usdc_swaps_mitigation_on_df.apply(js_mitigator_check, axis=1)
wbtc_usdc_swaps_mitigation_on_df

,id,token_in,token_out,token_in_amount,token_out_amount_min,token_out_amount,system_fee,mitigator_check_status,oracle_amount_out,out_amount_diff,slice_factor,slice_factor_curve,status,block_number,block_timestamp,transaction_timestamp,txd,sender,to,transaction_id,reserve_X_before,reserve_Y_before,k_before,price_X_cumulative_before,price_Y_cumulative_before,is_volatility_mitigator_on_before,reserve_X,reserve_Y,k,price_X_cumulative,price_Y_cumulative,is_volatility_mitigator_on,X_price,price_diff,js_mitigation_status
0,5,USDC,WBTC,1.534170e-01,0.0,0.000016,0.000614,CANT_CONSULT_ORACLE,NaN,NaN,NaN,NaN,SUCCESS,6348.0,2020-05-19 23:44:30,2020-05-19 23:44:23,0xba9a47ed6b7d293da7e7643f434055ff76bdf609274f...,0xf164fc0ec4e93095b804a4795bbe1e041497b92a,0x3da4491fed9f6d6b1391b2a02991c8d035d7716f,5,0.042839,4.156442e+02,1.780596e+01,0,0,True,0.042824,4.157970e+02,1.780600e+01,0,0,True,9709.485208,0.000733,
1,8,WBTC,USDC,2.501000e-04,0.0,2.351075,0.009404,CHECKED,2.428313,3.0,3.0,3.0,SUCCESS,15690.0,2020-05-21 14:40:00,2020-05-21 14:39:56,0xffff2870fd0aa2f909766844c95f53b83625854c925e...,0x96569f124f6a3b62093c7115f14f8005705ba48a,0x96569f124f6a3b62093c7115f14f8005705ba48a,8,0.010993,1.067336e+02,1.173307e+00,0,0,True,0.011243,1.043732e+02,1.173462e+00,1360573821,14,True,9283.431280,-0.043869,success
2,10,WBTC,USDC,1.047330e-03,0.0,9.536262,0.038145,CHECKED,9.722370,1.0,1.0,1.0,SUCCESS,22919.0,2020-05-22 20:47:15,2020-05-22 20:47:02,0x68e9800197efaafee864d33cf6167d12e4aa29a527c9...,0x96569f124f6a3b62093c7115f14f8005705ba48a,0x96569f124f6a3b62093c7115f14f8005705ba48a,10,0.111243,1.032669e+03,1.148771e+02,1360573821,14,True,0.112290,1.023094e+03,1.148836e+02,2367176485,26,True,9111.157903,-0.018512,success
3,11,USDC,WBTC,8.923190e-01,0.0,0.000097,0.003569,CHECKED,0.000097,NaN,NaN,NaN,SUCCESS,27053.0,2020-05-23 14:00:45,2020-05-23 14:00:42,0x33fbec2e6e1a3a04a1f5109f3a8809f32c7ad7e75c7d...,0x0d4743269fec611b422d0a209a460cd8cc0cc8b9,0x0d4743269fec611b422d0a209a460cd8cc0cc8b9,11,0.112290,1.023094e+03,1.148836e+02,2367176485,26,True,0.112193,1.023983e+03,1.148842e+02,2932159387,32,True,9126.946561,0.001733,success
4,12,WBTC,USDC,1.100000e-07,0.0,0.000994,0.000004,CHECKED,0.001003,NaN,NaN,NaN,SUCCESS,33681.0,2020-05-24 17:37:45,2020-05-24 17:37:32,0x273375cad4d5cf584b70af639f3cf07a53ba00a07216...,0xf164fc0ec4e93095b804a4795bbe1e041497b92a,0x211b6a1137bf539b2750e02b9e525cf5757a35ae,12,0.112193,1.023983e+03,1.148842e+02,2932159387,32,True,0.112194,1.023982e+03,1.148842e+02,3839560414,43,True,9126.928718,-0.000002,success
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71643,72786,WBTC,USDC,5.000000e-02,0.0,2161.702289,8.646809,CHECKED,2191.731004,1.0,1.0,1.0,SUCCESS,3950729.0,2022-04-04 18:39:45,2022-04-04 18:39:36,0x7e06f38d023fa0b907c90228cca0719ff24ba7924889...,0x1111111254fb6c44bac0bed2854e76f90643097d,0x3ab2b63d7edc734da546d2c60434494dadcd8d96,72786,27.032839,1.182706e+06,3.197190e+07,2032106998796,2712,True,27.082839,1.180536e+06,3.197226e+07,2032217250608,2712,True,43589.809642,-0.003678,success
71644,72787,WBTC,USDC,7.388149e-01,0.0,31044.336577,124.177346,CHECKED,32385.288297,4.0,3.0,3.0,SUCCESS,3950753.0,2022-04-04 18:45:45,2022-04-04 18:45:31,0x197b8153149e5467ff34e4833ff11fafb8ad3c344ec4...,0x0000000099cb7fc48a935bceb9f05bbae54e8987,0x0000000099cb7fc48a935bceb9f05bbae54e8987,72787,27.082839,1.180536e+06,3.197226e+07,2032217250608,2712,True,27.821653,1.149367e+06,3.197730e+07,2032232942940,2712,True,41311.968260,-0.052256,success
71645,72788,USDC,WBTC,2.369980e+03,0.0,0.056679,9.479920,CHECKED,0.054067,4.0,1.0,1.0,SUCCESS,3950753.0,2022-04-04 18:45:45,2022-04-04 18:45:31,0x9d43e381b2a260446d2bff4366dc9fa28640ff318a34...,0x68b3465833fb72a70ecdf485e0e4c7bd8665fc45,0xbb2b8038a1640196fbe3e38816f3e67cba72d940,72788,27.821653,1.149367e+06,3.197730e+07,2032232942940,2712,True,27.764975,1.151728e+06,3.197769e+07,2032232942940,2712,True,41481.318337,0.00

In [10]:
wbtc_usdc_swaps_mitigation_on_df[wbtc_usdc_swaps_mitigation_on_df['js_mitigation_status'] == "Error: Out value too far from Oracle"]

,id,token_in,token_out,token_in_amount,token_out_amount_min,token_out_amount,system_fee,mitigator_check_status,oracle_amount_out,out_amount_diff,slice_factor,slice_factor_curve,status,block_number,block_timestamp,transaction_timestamp,txd,sender,to,transaction_id,reserve_X_before,reserve_Y_before,k_before,price_X_cumulative_before,price_Y_cumulative_before,is_volatility_mitigator_on_before,reserve_X,reserve_Y,k,price_X_cumulative,price_Y_cumulative,is_volatility_mitigator_on,X_price,price_diff,js_mitigation_status
66,91,USDC,WBTC,31.840000,0.0,0.002410,NaN,CHECKED,2.994629e-03,21.0,24.0,96.0,BLOCKED_BY_VOLATILITY_MITIGATION,723999.0,2020-09-21 13:57:15,2020-09-21 13:57:04,0xca7ab3510e4e9c1425313598c138b91fd7ff09fc1b8e...,0x5bb456cd09d85156e182d2c7797eb49a43840187,0xe597410504cc3372210952c7ca3d8942023da277,91,0.012876,1.368994e+02,1.762679e+00,100203784052,1201,True,0.012876,1.368994e+02,1.762679e+00,101964504143,1216,True,10632.367700,0.0,Error: Out value too far from Oracle
67,92,WBTC,USDC,0.003264,0.0,27.465270,NaN,CHECKED,3.470543e+01,23.0,26.0,98.0,BLOCKED_BY_VOLATILITY_MITIGATION,724796.0,2020-09-21 17:16:30,2020-09-21 17:16:17,0xa83a8fcc0b6fdbcee0074a822dc76a9112f602ff891a...,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0xb4e16d0168e52d35cacd2c6185b44281ec28c9dc,92,0.012876,1.368994e+02,1.762679e+00,101964504143,1216,True,0.012876,1.368994e+02,1.762679e+00,102091614099,1218,True,10632.367700,0.0,Error: Out value too far from Oracle
65609,66703,USDC,WBTC,976323.861321,0.0,14.724979,NaN,CHECKED,2.241686e+01,41.0,51.0,98.0,BLOCKED_BY_VOLATILITY_MITIGATION,2862643.0,2021-09-27 20:58:15,2021-09-27 20:58:07,0x9bb156818f3af965f5d1ead14c1c55ed666e3d2eb36d...,0x00000000003b3cc22af3ae1eac0440bcee416b40,0x00000000003b3cc22af3ae1eac0440bcee416b40,66703,43.881509,1.913860e+06,8.398308e+07,1272010127889,2353,True,43.881509,1.913860e+06,8.398308e+07,1272010127889,2353,True,43614.278866,0.0,Error: Out value too far from Oracle
65610,66704,WBTC,USDC,12.689636,0.0,425965.986619,NaN,CHECKED,5.527246e+05,25.0,29.0,98.0,BLOCKED_BY_VOLATILITY_MITIGATION,2862643.0,2021-09-27 20:58:15,2021-09-27 20:58:07,0xbf0d923e72d3a107e8183b77e422c6efddb5d136137c...,0x00000000003b3cc22af3ae1eac0440bcee416b40,0x00000000003b3cc22af3ae1eac0440bcee416b40,66704,43.881509,1.913860e+06,8.398308e+07,1272010127889,2353,True,43.881509,1.913860e+06,8.398308e+07,1272010127889,2353,True,43614.278866,0.0,Error: Out value too far from Oracle
66005,67101,USDC,WBTC,557057.298176,0.0,12.267266,NaN,CHECKED,1.171552e+01,4.0,33.0,98.0,BLOCKED_BY_VOLATILITY_MITIGATION,2905850.0,2021-10-05 09:00:00,2021-10-05 09:00:00,0x09f234c943821301ec460ad7c2ba9b4ed651529d359e...,0x00000000008c4fb1c916e0c88fd4cc402d935e7d,0x00000000008c4fb1c916e0c88fd4cc402d935e7d,67101,50.239569,1.707081e+06,8.576303e+07,1301300736541,2367,True,50.239569,1.707081e+06,8.576303e+07,1301300736541,2367,True,33978.821002,0.0,Error: Out value too far from Oracle
67843,68952,WBTC,USDC,15.779159,0.0,644612.521068,NaN,CHECKED,1.017970e+06,44.0,52.0,98.0,BLOCKED_BY_VOLATILITY_MITIGATION,3146445.0,2021-11-16 03:28:45,2021-11-16 03:28:33,0x34ccc48a1b7898a258159872b583775cd97ebbe7cbe9...,0x00000000003b3cc22af3ae1eac0440bcee416b40,0x00000000003b3cc22af3ae1eac0440bcee416b40,68952,30.556905,1.905537e+06,5.822731e+07,1500967900657,2434,True,30.556905,1.905537e+06,5.822731e+07,1500967900657,2434,True,62360.272135,0.0,Error: Out value too far from Oracle
67844,68953,USDC,WBTC,515070.490141,0.0,6.450776,NaN,CHECKED,7.985159e+00,21.0,27.0,98.0,BLOCKED_BY_VOLATILITY_MITIGATION,3146445.0,2021-11-16 03:28:45,2021-11-16 03:28:33,0x891cf339a78154fac5a8974e17f26ba6b3304a0dc5fa...,0x00000000003b3cc22af3ae1eac0440bcee416b40,0x4585fe77225b41b697c938b018e2ac67ac5a20c0,68953,30.556905,1.905537e+06,5.822731e+07,1500967900657,2434,True,30.556905,1.905537e+06,5.822731e+07,1500967900657,2434,True,62360.272135,0.0,Error: Out value too far from Oracle
68212,69323,USDC,WBTC,410078.298461,0.0,5.826826,NaN,CHECKED,7.194719e+00,21.0,23.0,92.0,BLOCKED_

In [11]:
wbtc_usdc_swaps_mitigation_on_df[wbtc_usdc_swaps_mitigation_on_df['status'] == "BLOCKED_BY_VOLATILITY_MITIGATION"]

,id,token_in,token_out,token_in_amount,token_out_amount_min,token_out_amount,system_fee,mitigator_check_status,oracle_amount_out,out_amount_diff,slice_factor,slice_factor_curve,status,block_number,block_timestamp,transaction_timestamp,txd,sender,to,transaction_id,reserve_X_before,reserve_Y_before,k_before,price_X_cumulative_before,price_Y_cumulative_before,is_volatility_mitigator_on_before,reserve_X,reserve_Y,k,price_X_cumulative,price_Y_cumulative,is_volatility_mitigator_on,X_price,price_diff,js_mitigation_status
66,91,USDC,WBTC,31.840000,0.0,0.002410,NaN,CHECKED,2.994629e-03,21.0,24.0,96.0,BLOCKED_BY_VOLATILITY_MITIGATION,723999.0,2020-09-21 13:57:15,2020-09-21 13:57:04,0xca7ab3510e4e9c1425313598c138b91fd7ff09fc1b8e...,0x5bb456cd09d85156e182d2c7797eb49a43840187,0xe597410504cc3372210952c7ca3d8942023da277,91,0.012876,1.368994e+02,1.762679e+00,100203784052,1201,True,0.012876,1.368994e+02,1.762679e+00,101964504143,1216,True,10632.367700,0.0,Error: Out value too far from Oracle
67,92,WBTC,USDC,0.003264,0.0,27.465270,NaN,CHECKED,3.470543e+01,23.0,26.0,98.0,BLOCKED_BY_VOLATILITY_MITIGATION,724796.0,2020-09-21 17:16:30,2020-09-21 17:16:17,0xa83a8fcc0b6fdbcee0074a822dc76a9112f602ff891a...,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0xb4e16d0168e52d35cacd2c6185b44281ec28c9dc,92,0.012876,1.368994e+02,1.762679e+00,101964504143,1216,True,0.012876,1.368994e+02,1.762679e+00,102091614099,1218,True,10632.367700,0.0,Error: Out value too far from Oracle
65609,66703,USDC,WBTC,976323.861321,0.0,14.724979,NaN,CHECKED,2.241686e+01,41.0,51.0,98.0,BLOCKED_BY_VOLATILITY_MITIGATION,2862643.0,2021-09-27 20:58:15,2021-09-27 20:58:07,0x9bb156818f3af965f5d1ead14c1c55ed666e3d2eb36d...,0x00000000003b3cc22af3ae1eac0440bcee416b40,0x00000000003b3cc22af3ae1eac0440bcee416b40,66703,43.881509,1.913860e+06,8.398308e+07,1272010127889,2353,True,43.881509,1.913860e+06,8.398308e+07,1272010127889,2353,True,43614.278866,0.0,Error: Out value too far from Oracle
65610,66704,WBTC,USDC,12.689636,0.0,425965.986619,NaN,CHECKED,5.527246e+05,25.0,29.0,98.0,BLOCKED_BY_VOLATILITY_MITIGATION,2862643.0,2021-09-27 20:58:15,2021-09-27 20:58:07,0xbf0d923e72d3a107e8183b77e422c6efddb5d136137c...,0x00000000003b3cc22af3ae1eac0440bcee416b40,0x00000000003b3cc22af3ae1eac0440bcee416b40,66704,43.881509,1.913860e+06,8.398308e+07,1272010127889,2353,True,43.881509,1.913860e+06,8.398308e+07,1272010127889,2353,True,43614.278866,0.0,Error: Out value too far from Oracle
66005,67101,USDC,WBTC,557057.298176,0.0,12.267266,NaN,CHECKED,1.171552e+01,4.0,33.0,98.0,BLOCKED_BY_VOLATILITY_MITIGATION,2905850.0,2021-10-05 09:00:00,2021-10-05 09:00:00,0x09f234c943821301ec460ad7c2ba9b4ed651529d359e...,0x00000000008c4fb1c916e0c88fd4cc402d935e7d,0x00000000008c4fb1c916e0c88fd4cc402d935e7d,67101,50.239569,1.707081e+06,8.576303e+07,1301300736541,2367,True,50.239569,1.707081e+06,8.576303e+07,1301300736541,2367,True,33978.821002,0.0,Error: Out value too far from Oracle
67843,68952,WBTC,USDC,15.779159,0.0,644612.521068,NaN,CHECKED,1.017970e+06,44.0,52.0,98.0,BLOCKED_BY_VOLATILITY_MITIGATION,3146445.0,2021-11-16 03:28:45,2021-11-16 03:28:33,0x34ccc48a1b7898a258159872b583775cd97ebbe7cbe9...,0x00000000003b3cc22af3ae1eac0440bcee416b40,0x00000000003b3cc22af3ae1eac0440bcee416b40,68952,30.556905,1.905537e+06,5.822731e+07,1500967900657,2434,True,30.556905,1.905537e+06,5.822731e+07,1500967900657,2434,True,62360.272135,0.0,Error: Out value too far from Oracle
67844,68953,USDC,WBTC,515070.490141,0.0,6.450776,NaN,CHECKED,7.985159e+00,21.0,27.0,98.0,BLOCKED_BY_VOLATILITY_MITIGATION,3146445.0,2021-11-16 03:28:45,2021-11-16 03:28:33,0x891cf339a78154fac5a8974e17f26ba6b3304a0dc5fa...,0x00000000003b3cc22af3ae1eac0440bcee416b40,0x4585fe77225b41b697c938b018e2ac67ac5a20c0,68953,30.556905,1.905537e+06,5.822731e+07,1500967900657,2434,True,30.556905,1.905537e+06,5.822731e+07,1500967900657,2434,True,62360.272135,0.0,Error: Out value too far from Oracle
68212,69323,USDC,WBTC,410078.298461,0.0,5.826826,NaN,CHECKED,7.194719e+00,21.0,23.0,92.0,BLOCKED_

In [12]:
# wbtc_usdc_swaps_mitigation_on_df.drop(columns=['id'], inplace=True)

In [13]:
# wbtc_usdc_swaps_mitigation_on_df["reserve_X_before"] = wbtc_usdc_swaps_mitigation_on_df["reserve_X_before"].apply(lambda x: big_numbers.expand_to_18_decimals(x))
# wbtc_usdc_swaps_mitigation_on_df["reserve_Y_before"] = wbtc_usdc_swaps_mitigation_on_df["reserve_Y_before"].apply(lambda x: big_numbers.expand_to_18_decimals(x))#
# wbtc_usdc_swaps_mitigation_on_df["token_in_amount"] = wbtc_usdc_swaps_mitigation_on_df["token_in_amount"].apply(lambda x: big_numbers.expand_to_18_decimals(x))
# wbtc_usdc_swaps_mitigation_on_df["token_out_amount"] = wbtc_usdc_swaps_mitigation_on_df["token_out_amount"].apply(lambda x: big_numbers.expand_to_18_decimals(x))
# wbtc_usdc_swaps_mitigation_on_df["oracle_amount_out"] = wbtc_usdc_swaps_mitigation_on_df["oracle_amount_out"].apply(lambda x: big_numbers.expand_to_18_decimals(x))
# wbtc_usdc_swaps_mitigation_on_df.to_csv("swaps_normalized.csv")

In [14]:
# wbtc_usdc_swaps_mitigation_on_df[wbtc_usdc_swaps_mitigation_on_df['status'] == "BLOCKED_BY_VOLATILITY_MITIGATION"]